In [1]:
%load_ext autoreload
%autoreload 2


In [2]:
from notte.browser.driver import BrowserDriver


driver = BrowserDriver()

await driver.start()

# driver.stop()


In [3]:
from playwright.async_api import async_playwright


url = "https://www.google.com/travel/flights"
async def record_trace(url: str):
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=False)
        context = await browser.new_context()

        # Start tracing
        await context.tracing.start(screenshots=True, snapshots=True)

        page = await context.new_page()
        await page.goto(url)

        # Perform actions
        await page.get_by_role("combobox", name="Where from?").fill("London")
        await page.get_by_role("combobox", name="Where to?").fill("Paris")
        await page.get_by_role("textbox", name="Departure").fill("2025-01-01")
        await page.get_by_role("textbox", name="Return").fill("2025-02-01")
        await page.keyboard.press("Enter")
        await page.wait_for_timeout(1000)
        await page.get_by_role("button", name="Search").click()
        await page.wait_for_timeout(3000)
        # Stop tracing and save the trace file
        await context.tracing.stop(path="trace.zip")
        print("Trace recorded to trace.zip")

        browser.close()


In [ ]:
import os
from urllib.parse import urlparse
from playwright.async_api import async_playwright


async def save_website_snapshot_old(url: str        ):
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=False)
        context = await browser.new_context()

        snapshot_dir = "./website_snapshot"
        os.makedirs(snapshot_dir, exist_ok=True)

        page = await context.new_page()

        # Intercept and save all resources
        async def save_response(route):
            url = route.request.url
            try:
                # Create file path based on the URL structure
                file_path = os.path.join(snapshot_dir, url.replace("https://", "").replace("/", "_"))
                response = await route.fetch()
                content = await response.body()
                with open(file_path, "wb") as f:
                    f.write(content)
                print(f"Saved: {file_path}")
            except Exception as e:
                print(f"Failed to save resource: {url}, Error: {e}")
            await route.continue_()

        await context.route("**/*", save_response)

        await page.goto("https://www.google.com/travel/flights")
        print("Snapshot saved to website_snapshot/")
        await browser.close()
        
import os
from playwright.sync_api import sync_playwright

async def save_website_snapshot_2(url: str):
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=False)
        context = await browser.new_context()

        snapshot_dir = "./website_snapshot"
        os.makedirs(snapshot_dir, exist_ok=True)

        page = await context.new_page()

        # Intercept and save all resources
        async def save_response(route):
            url = route.request.url
            try:
                # Create a file path based on the URL structure
                if route.request.url == page.url:  # Main page content
                    file_path = os.path.join(snapshot_dir, "index.html")
                else:
                    # Clean up URL to create a valid filename
                    filename = (
                        route.request.url.replace("https://", "")
                        .replace("http://", "")
                        .replace("/", "_")
                        .replace("?", "_")
                        .replace("&", "_")
                    )
                    file_path = os.path.join(snapshot_dir, filename)

                response = await route.fetch()
                content = await response.body()
                with open(file_path, "wb") as f:
                    f.write(content)
                print(f"Saved: {file_path}")
            except Exception as e:
                print(f"Failed to save resource: {route.request.url}, Error: {e}")
            await route.continue_()

        await context.route("**/*", save_response)

        # Navigate to the website and save the main page
        await page.goto(url)
        print("Snapshot saved to website_snapshot/")

        await browser.close()





async def save_website_snapshot(url: str):
    async with async_playwright() as p:
        browser = await p.chromium.launch()
        context = await browser.new_context()

        # Directory to store the snapshot
        snapshot_dir = "./website_snapshot"
        os.makedirs(snapshot_dir, exist_ok=True)

        page = await context.new_page()

        # Navigate to the website
        await page.goto(url)  # Replace with the URL of the target website

        # Save the main page content as `index.html`
        main_html_path = os.path.join(snapshot_dir, "index.html")
        with open(main_html_path, "w", encoding="utf-8") as f:
            f.write(await page.content())
        print(f"Saved main page as: {main_html_path}")

        # Intercept and save all additional resources (CSS, JS, images, etc.)
        async def save_resource(route):
            url = route.request.url
            parsed_url = urlparse(url)
            if not parsed_url.path:
                await route.continue_()
                return

            # Clean up URL path for a valid filename
            resource_name = parsed_url.path.strip("/").replace("/", "_")
            if not resource_name:
                resource_name = "index"  # Fallback for empty paths
            if "." not in resource_name:
                resource_name += ".bin"  # Default extension for binary resources

            file_path = os.path.join(snapshot_dir, resource_name)
            try:
                # Fetch and save the resource
                response = await route.fetch()
                body = await response.body()
                with open(file_path, "wb") as f:
                    f.write(body)
                print(f"Saved resource: {file_path}")
            except Exception as e:
                print(f"Failed to save resource {url}: {e}")

            await route.continue_()

        # Intercept network requests
        await context.route("**/*", save_resource)

        # Reload the page to trigger resource downloads
        await page.reload()

        await browser.close()



# await record_trace("https://www.google.com/travel/flights")
await save_website_snapshot("https://www.google.com/travel/flights")


In [ ]:
await record_trace("http://localhost:8080")

# Final

In [ ]:
from pathlib import Path
from urllib.parse import urlparse
from playwright.async_api import async_playwright, Page
from collections.abc import Callable, Awaitable

async def save_snapshot(page, snapshot_path: Path):
    """
    Saves the current page as `index.html` and downloads all additional resources.
    Args:
        page: The Playwright Page object.
        snapshot_path: Path to save the snapshot.
    """
    snapshot_path.mkdir(parents=True, exist_ok=True)

    # Save the main page content as `index.html`
    main_html_path = snapshot_path / "index.html"
    with main_html_path.open("w", encoding="utf-8") as f:
        f.write(await page.content())
    print(f"Saved main page as: {main_html_path}")

    # Intercept and save all additional resources (CSS, JS, images, etc.)
    async def save_resource(route):
        url = route.request.url
        parsed_url = urlparse(url)
        if not parsed_url.path:
            await route.continue_()
            return

        # Clean up URL path for a valid filename
        resource_name = parsed_url.path.strip("/").replace("/", "_")
        if not resource_name:
            resource_name = "index"  # Fallback for empty paths
        if "." not in resource_name:
            resource_name += ".bin"  # Default extension for binary resources

        file_path = snapshot_path / resource_name
        try:
            # Fetch and save the resource
            response = await route.fetch()
            with file_path.open("wb") as f:
                f.write(await response.body())
            print(f"Saved resource: {file_path}")
        except Exception as e:
            print(f"Failed to save resource {url}: {e}")

        await route.continue_()

    # Intercept network requests for this snapshot
    await page.context.route("**/*", save_resource)

    # Reload the page to trigger resource downloads
    await page.reload()

async def perform_trace(
    url: str,
    steps: list[Callable[[Page], Awaitable[None]]],
    snapshot_dir: str = "./website_snapshots",
):
    snapshot_dir = Path(snapshot_dir)
    snapshot_dir.mkdir(parents=True, exist_ok=True)
    
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=False)  # Set headless=True if you don't want a visible browser
        context = await browser.new_context()
        page = await context.new_page()

        # Directory to store snapshots
       

        # Navigate to the initial page
        await page.goto(url)  # Replace with your target website
        await save_snapshot(page, snapshot_dir / "step0")

        # Example actions (trace steps)
        # 1. Click a button (replace with actual selector)
        for i, step in enumerate(steps):  
            try:
                await step(page)
                await save_snapshot(page, snapshot_dir / f"step{i+1}")
            except Exception as e:
                print(f"Step {i+1} failed: {e}")
        # Add more steps as needed
        await browser.close()


# url = "https://www.google.com/travel/flights"
url = "https://example.com"
dir_name = 'test_tracing_2'

steps = [
    lambda page: page.click("text=More information"),
    # lambda page: page.fill("#search-box", "Playwright testing"),
    # lambda page: page.press("#search-box", "Enter"),
]
# Run the async function
await perform_trace(url, steps, snapshot_dir=dir_name)


In [ ]:
from pathlib import Path
from http.server import SimpleHTTPRequestHandler
from socketserver import TCPServer
import threading
import time
from IPython.display import IFrame, display




class SnapshotServer:
    def __init__(self, snapshot_dir, port=8080):
        """
        Initialize the snapshot server.

        Args:
            snapshot_dir (str): Path to the directory containing snapshots.
            port (int): Port for the HTTP server.
        """
        self.snapshot_dir = Path(snapshot_dir)
        self.port = port
        self.httpd = None
        self.thread = None
        self.snapshots = sorted(self.snapshot_dir.iterdir())
        self.current_index = 0

        if not self.snapshots:
            raise ValueError(f"No snapshots found in {snapshot_dir}")

    def start_server(self):
        """
        Start the HTTP server in a separate thread.
        """
        self.snapshot_dir = self.snapshot_dir.resolve()

        def run_server():
            handler = SimpleHTTPRequestHandler
            self.httpd = TCPServer(("0.0.0.0", self.port), handler)
            print(f"Serving HTTP on port {self.port}...")
            self.httpd.serve_forever()

        self.thread = threading.Thread(target=run_server)
        self.thread.daemon = True
        self.thread.start()
        time.sleep(1)  # Allow the server to start

    def stop_server(self):
        """
        Stop the HTTP server.
        """
        if self.httpd:
            self.httpd.shutdown()
            self.httpd.server_close()
            print("HTTP server stopped.")
        if self.thread:
            self.thread.join()

    def step(self, forward=True):
        """
        Move to the next or previous snapshot and render it.

        Args:
            forward (bool): Move forward if True, backward if False.
        """
        if forward:
            if self.current_index < len(self.snapshots) - 1:
                self.current_index += 1
            else:
                print("Already at the last snapshot.")
        else:
            if self.current_index > 0:
                self.current_index -= 1
            else:
                print("Already at the first snapshot.")

# Example usage
server = SnapshotServer(dir_name, port=8081)

# Start the server
server.start_server()

# Step forward through snapshots
server.step(forward=True)

# Step backward through snapshots
server.step(forward=False)

# Stop the server when done
server.stop_server()


In [22]:
from pathlib import Path

Path('test_tracing').mkdir(parents=True, exist_ok=True)


In [ ]:
Path('test_tracing').absolute()